# BLIP VQA Labeled Test
## True/False 비율 기반 랜드마크 인식 평가

기존의 YES 비율 평가에서 개선하여, 각 질문마다 예상 답변(yes/no)을 라벨링하고
BLIP 모델의 답변이 예상과 일치하는지 True/False로 평가합니다.

In [ ]:
import torch
from transformers import BlipProcessor, BlipForQuestionAnswering
from PIL import Image
import glob
import os
import json
from tqdm.auto import tqdm

# GPU 사용 설정
device = "cuda" if torch.cuda.is_available() else "cpu"

# 모델 로드
model_name = "ybelkada/blip-vqa-base"
processor = BlipProcessor.from_pretrained(model_name)
model = BlipForQuestionAnswering.from_pretrained(model_name).to(device)

print(f"BLIP 모델 로드 완료. (사용 장치: {device})")

In [ ]:
# 로컬/서버에서 실행할 땐 아래 코드 주석처리 하세요
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
import os

# --- 1. 환경별 프로젝트 루트 경로 정의 ---
# (경로 1: Google Drive Mount Colab)
COLAB_DRIVE_ROOT = "/content/drive/Othercomputers/내 컴퓨터/데이콘 출판마을 프로젝트/github"
# (경로 2: "서버" 또는 다른 Colab 환경)
SERVER_ROOT = "/content/group5_project"

# --- 2. 환경 감지 및 PROJECT_ROOT 설정 ---
if os.path.exists(COLAB_DRIVE_ROOT):
    PROJECT_ROOT = COLAB_DRIVE_ROOT
    print(f"환경 감지: Google Drive Colab")
elif os.path.exists(SERVER_ROOT):
    PROJECT_ROOT = SERVER_ROOT
    print(f"환경 감지: Server (group5_project)")

# --- 3. 최종 DATA_DIR 설정 ---
DATA_DIR = os.path.join(PROJECT_ROOT, "data")
print(f"최종 DATA_DIR: {DATA_DIR}")

# --- 4. 파일 경로 설정 ---
LANDMARK_QA_LABELED_FILE = os.path.join(DATA_DIR, "landmark_qa_labeled.json")
print(f"JSON 파일 경로: {LANDMARK_QA_LABELED_FILE}")

In [ ]:
# landmark_qa_labeled.json 로드
with open(LANDMARK_QA_LABELED_FILE, 'r', encoding='utf-8') as f:
    labeled_qa_data = json.load(f)
    print(f"Labeled Q&A data loaded from '{LANDMARK_QA_LABELED_FILE}'.")

# 데이터 확인
for landmark, qa_list in labeled_qa_data.items():
    print(f"\n{landmark}: {len(qa_list)}개 질문")
    print(f"  예시: {qa_list[0]}")

In [ ]:
# 테스트할 랜드마크 선택
LANDMARK_NAME = "네모탑"  # "피노키오", "네모탑", "지혜의숲 조각상" 중 선택
SUCCESS_THRESHOLD = 0.70  # 성공 판정 임계값 (70%)

landmark_dir = os.path.join(DATA_DIR, LANDMARK_NAME)
print(f"테스트 랜드마크: {LANDMARK_NAME}")
print(f"이미지 폴더: {landmark_dir}")
print(f"성공 임계값: {SUCCESS_THRESHOLD:.0%}")

In [ ]:
# --- 1. 이미지 파일 찾기 ---
image_extensions = ["*.jpg", "*.jpeg", "*.png", "*.webp", "*.jfif"]
image_files = []
for ext in image_extensions:
    image_files.extend(glob.glob(os.path.join(landmark_dir, ext)))

if not image_files:
    print(f"경고: '{landmark_dir}' 폴더에서 이미지를 찾을 수 없습니다. 경로를 확인해주세요.")
else:
    print(f"'{LANDMARK_NAME}' 폴더에서 총 {len(image_files)}개의 이미지를 찾았습니다.")

# --- 2. labeled Q&A 리스트 가져오기 ---
labeled_questions = labeled_qa_data.get(LANDMARK_NAME, [])
total_questions_count = len(labeled_questions)

if total_questions_count == 0:
    print(f"경고: '{LANDMARK_NAME}'에 대한 질문 리스트가 비어있습니다.")
else:
    print(f"'{LANDMARK_NAME}'에 대한 질문: {total_questions_count}개")
    yes_label_count = sum(1 for _, label in labeled_questions if label == "yes")
    no_label_count = sum(1 for _, label in labeled_questions if label == "no")
    print(f"  - Positive (yes): {yes_label_count}개")
    print(f"  - Negative (no): {no_label_count}개")

In [ ]:
# --- 3. VQA 실행 및 True/False 평가 ---
image_results_list = []  # 이미지별 결과를 저장할 리스트

if total_questions_count > 0 and image_files:
    # tqdm을 사용하여 진행률 표시
    for img_path in tqdm(image_files, desc=f"'{LANDMARK_NAME}' 이미지 처리 중"):
        try:
            image = Image.open(img_path).convert("RGB")
            img_name = os.path.basename(img_path)

            print("\n" + "="*50)
            print(f"{img_name} 처리 중")
            print("="*50)

            # 이미지 표시 (Colab에서만)
            try:
                from IPython.display import display
                display(image.resize((300, 300)))
            except:
                pass

            true_count = 0
            false_count = 0
            false_questions_list = []  # False인 질문들 저장

            pixel_values = processor(images=image, return_tensors="pt").pixel_values.to(device)

            for question, expected_label in labeled_questions:
                inputs = processor(text=question, return_tensors="pt").to(device)

                out = model.generate(
                    pixel_values=pixel_values,
                    input_ids=inputs.input_ids,
                    attention_mask=inputs.attention_mask,
                    max_new_tokens=10
                )
                answer = processor.decode(out[0], skip_special_tokens=True).strip().lower()

                # True/False 판정
                is_correct = (answer == expected_label)
                status_icon = "✅" if is_correct else "❌"

                # 상세 로그 출력
                print(f"  {status_icon} Q: {question}")
                print(f"     Expected: {expected_label}, Got: {answer}")

                if is_correct:
                    true_count += 1
                else:
                    false_count += 1
                    false_questions_list.append({
                        "question": question,
                        "expected": expected_label,
                        "got": answer
                    })

            # --- 결과 계산 및 저장 ---
            true_ratio = (true_count / total_questions_count) if total_questions_count > 0 else 0.0
            is_success = true_ratio >= SUCCESS_THRESHOLD

            print(f"\n  [결과] True: {true_count}, False: {false_count}")
            print(f"  정확도: {true_ratio:.2%}")
            print(f"  성공 여부: {'✅ PASS' if is_success else '❌ FAIL'}")

            image_results_list.append({
                "image": img_name,
                "true_count": true_count,
                "false_count": false_count,
                "true_ratio": true_ratio,
                "is_success": is_success,
                "false_questions": false_questions_list,
                "total_questions": total_questions_count
            })

        except Exception as e:
            print(f"{img_path} 처리 중 오류 발생: {e}")

print("\n\n" + "="*50)
print(f"VQA 처리 완료. 'image_results_list' 변수에 {len(image_results_list)}개 결과 저장됨.")
print("="*50)

In [ ]:
# --- 최종 요약 리스트 출력 ---

if 'image_results_list' not in locals() or not image_results_list:
    print("❌ 오류: 'image_results_list' 변수를 찾을 수 없습니다.")
    print("이전 셀을 먼저 실행하여 VQA 처리를 완료해주세요.")
else:
    print("\n" + "="*50)
    print(f"    '{LANDMARK_NAME}' VQA 이미지별 최종 평가 결과")
    print("="*50)

    total_accuracy = 0
    success_count = 0

    for result in image_results_list:
        print(f"\n이미지: {result['image']}")
        print(f"  - True  : {result['true_count']} / {result['total_questions']}")
        print(f"  - False : {result['false_count']}")
        print(f"  - 정확도: {result['true_ratio']:.2%}")
        print(f"  - 성공 여부: {'✅ PASS' if result['is_success'] else '❌ FAIL'}")

        # False 질문 목록 출력
        if result['false_questions']:
            print(f"\n  ❌ False 질문 목록 ({len(result['false_questions'])}개):")
            for i, fq in enumerate(result['false_questions'], 1):
                print(f"    {i}. Q: {fq['question']}")
                print(f"       Expected: {fq['expected']}, Got: {fq['got']}")

        print("-" * 50)

        total_accuracy += result['true_ratio']
        if result['is_success']:
            success_count += 1

    # 전체 통계
    avg_accuracy = total_accuracy / len(image_results_list)
    success_rate = success_count / len(image_results_list)

    print("\n" + "="*50)
    print("전체 통계")
    print("="*50)
    print(f"평균 정확도: {avg_accuracy:.2%}")
    print(f"성공 이미지: {success_count} / {len(image_results_list)} ({success_rate:.2%})")
    print(f"성공 임계값: {SUCCESS_THRESHOLD:.0%}")

In [ ]:
# --- (선택) 결과를 JSON 파일로 저장 ---
import json

output_file = os.path.join(PROJECT_ROOT, "tests", f"blip_results_{LANDMARK_NAME}.json")

with open(output_file, 'w', encoding='utf-8') as f:
    json.dump({
        "landmark": LANDMARK_NAME,
        "threshold": SUCCESS_THRESHOLD,
        "total_images": len(image_results_list),
        "success_count": success_count,
        "average_accuracy": avg_accuracy,
        "results": image_results_list
    }, f, ensure_ascii=False, indent=2)

print(f"결과가 '{output_file}'에 저장되었습니다.")